In [1]:
# Import main libraries

import urllib.request
import time

class StopExecution(Exception):
    pass

In [2]:
# Read the Sudoku grid from the provided source
def readSudoku(url):

  # Get the whole content of the source
  with urllib.request.urlopen(url) as response:
    content = response.read().decode("utf-8")

  # Filter out rows (a line is a row)
  rows = [line for line in content.split('\r\n')]

  # Convert each row in a list of integers
  # Getting all the elemets on Sudoku grid
  grid = [[],[],[],[],[],[],[],[],[]]
  for row in range(0,9):
    grid[row] = [int(x) for x in str(rows[row])]

  return grid

In [3]:
# This function provides with a meaningful representation of the Sudoku grid
# The all grid is divided in 9 sub-grids

def print_grid():
  global grid
  for row in range(9):
    if row % 3 == 0 and row != 0:
      print("- - - - - - - - - - - -")
    for col in range(9):
      if col % 3 == 0 and col != 0:
        print(" | ", end="")
      if col == 8:
        print(grid[row][col])
      else:
        print(f'{str(grid[row][col])} ', end="")

In [4]:
# This function counts the number of intial given numbers and tell us
# if the given Sudoku puzzle has a unique solution or not.

def well_formed():
  global grid
  count = 0
  for row in range(0,9):
    for col in range(0,9):
      if grid[row][col] != 0:
        count +=1

  if count >= 17:
    print(f'\nThe puzzle has {count} clues.\nUnique solution is expected!')
  else:
    print(f'\nThe puzzle has {count} clues.\nMultiple solutions are expected!')

In [5]:
# This function is used to satisfy the Sudoku's rules
# It takes a number and it's possible position (p_row, p_col)
# and checks if this number can be inserted into that position without violating
# the Sudoku's rules or not.

def allowed(p_row,p_col,num):
  global grid

  # 1) Check for row conflict
  for col in range(0,9):
    if grid[p_row][col] == num :
      return False

  # 2) Check for column conflict
  for row in range(0,9):
    if grid[row][p_col] == num :
      return False

  # 3) Check for sub-grid conflict
  # (row_0,col_0) starting position of the sub-grid
  row_0 = (p_row // 3) * 3
  col_0 = (p_col // 3) * 3
  for row in range(0,3):
    for col in range(0,3):
      if grid[row_0 + row][col_0 + col] == num :
        return False

  # If there is not any conflict, return True
  return True

In [6]:
# This function finds the most constrained empty cell.
# The cell on the grid which has the fewest allowed(possible) numbers in it.
# Process: 1) For each empty cell create a list containing
#             the cell coordinates and the allowed numbers in it.
#          2) Return the shortest list (best candidate)

def find_candidate():
  global grid

  list_of_candidates = []
  for row in range(9):
    for col in range(9):
      candidate = []

      # find a candidate
      if grid[row][col] == 0:
        # add its coordinates
        candidate +=[row,col]
        # add its allowed numbers
        for num in range(1,10):
          if allowed(row,col,num):
            candidate.append(num)

        # add candidate to a comparison list
        list_of_candidates.append(candidate)

  # Handle the situation when the grid is filled
  if not list_of_candidates:
    return 0

  # Find the best candidate
  min = list_of_candidates[0]
  for i in range(1, len(list_of_candidates)):
    if len(list_of_candidates[i]) < len(min):
      min = list_of_candidates[i]

  return min

The Sudoku solving algorithm using the clever (DFS) brute force with backtracking:
  1. Choose the best empty cell in the Sudoku grid.
  2. Try each allowed number in that cell.
  3. If the chosen number is valid (satisfies Sudoku rules), move to the next best empty cell and repeat steps 1-3 recursively.
  4. If no valid number is found for a cell, backtrack to the previous cell (decision node) and try a different number.
  5. Repeat steps 1-4 until the entire Sudoku is filled.


The solve() implements the clever DFS brute force with backtracking using recrusion.


In [7]:
def solve():
  global grid
  global start_time
  global caller
  caller +=1
  # global timer

  # 1) Choose the best empty cell in the Sudoku grid.
  candidate = find_candidate()

  # When the grid is filled, print the grid
  if candidate == 0:
    end_time = time.time()
    print_grid()
    print(f'\n{end_time-start_time}\n')
    print(f'\n{caller}\n')
    #timer += end_time-start_time

    # By backtracking you can find other solutions if they exist.
    # It will try other numbers, if allowed.
    user_input = input("Press 'Enter' to look for other solutions.\n(Type 'Stop' to exit): ")
    if user_input.lower() == "stop":
        raise StopExecution()

  # 4) When dead-end happens, backtrack to the previous call (decision node).
  elif len(candidate) == 2:
    return

  # 2,3,5) Solve the grid using the allowed numbers
  else:
    for i in range(2,len(candidate)):                 # 2)
      grid[candidate[0]][candidate[1]] = candidate[i] # 3)
      solve()                                         # 5)
      grid[candidate[0]][candidate[1]] = 0

In [8]:
grid = readSudoku("https://uchida.cis.k.hosei.ac.jp/2015/optBprob-hard2.txt")

In [ ]:
grid[0][5] = 0

In [ ]:
grid = [
    [8,0,0,0,0,0,0,0,0],
    [0,0,3,6,0,0,0,0,0],
    [0,7,0,0,9,0,2,0,0],
    [0,5,0,0,0,7,0,0,0],
    [0,0,0,0,4,5,7,0,0],
    [0,0,0,1,0,0,0,3,0],
    [0,0,1,0,0,0,0,6,8],
    [0,0,8,5,0,0,0,1,0],
    [0,9,0,0,0,0,4,0,0]
]

In [17]:
grid = [
    [3, 0, 0, 7, 0, 0, 0, 0, 0],
    [0, 9, 0, 0, 4, 0, 7, 0, 0],
    [0, 0, 0, 6, 2, 1, 0, 8, 0],
    [8, 6, 0, 0, 0, 0, 2, 0, 0],
    [9, 0, 0, 0, 0, 0, 0, 0, 3],
    [0, 0, 4, 0, 0, 0, 0, 5, 8],
    [0, 7, 0, 2, 9, 5, 0, 0, 0],
    [0, 0, 3, 0, 8, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 6, 0, 0, 7]
]

In [8]:
grid = [
    [1,0,0,0,4,0,0,9,0],
    [0,0,7,0,6,0,0,0,0],
    [0,8,0,7,0,9,3,0,0],
    [0,0,2,3,0,7,0,1,0],
    [0,0,0,6,0,0,0,0,0],
    [0,4,0,0,0,0,0,0,2],
    [5,0,0,0,0,0,8,0,0],
    [0,0,4,2,0,1,0,3,0],
    [0,0,0,0,9,0,0,0,0]
]

In [9]:
print_grid()
well_formed()

0 0 0  | 0 0 1  | 0 3 0
0 9 5  | 6 0 0  | 0 0 0
0 8 0  | 0 0 0  | 0 0 0
- - - - - - - - - - - -
3 0 0  | 0 9 0  | 0 0 0
0 0 0  | 8 0 0  | 0 0 0
4 0 0  | 0 0 0  | 6 2 0
- - - - - - - - - - - -
0 0 0  | 0 0 0  | 0 0 7
0 0 0  | 0 0 0  | 5 0 8
2 0 0  | 0 0 4  | 0 0 0

The puzzle has 17 clues.
Unique solution is expected!


In [30]:
print(find_candidate())

[0, 0, 6, 7]


In [10]:
try:
    caller = 0
    #timer
    start_time = time.time()
    solve()
except StopExecution:
    print("Execution stopped.")

7 2 4  | 9 5 1  | 8 3 6
1 9 5  | 6 3 8  | 4 7 2
6 8 3  | 2 4 7  | 9 5 1
- - - - - - - - - - - -
3 1 2  | 4 9 6  | 7 8 5
5 6 9  | 8 7 2  | 3 1 4
4 7 8  | 3 1 5  | 6 2 9
- - - - - - - - - - - -
8 3 1  | 5 6 9  | 2 4 7
9 4 7  | 1 2 3  | 5 6 8
2 5 6  | 7 8 4  | 1 9 3

25.36378526687622


37493

Press 'Enter' to look for other solutions.
(Type 'Stop' to exit): 


In [ ]:
urls =['https://uchida.cis.k.hosei.ac.jp/2015/optBprob-easy.txt','https://uchida.cis.k.hosei.ac.jp/2015/optBprob-medium.txt',
       'https://uchida.cis.k.hosei.ac.jp/2015/optBprob-hard1.txt','https://uchida.cis.k.hosei.ac.jp/2015/optBprob-hard2.txt',
       'https://uchida.cis.k.hosei.ac.jp/2015/optBprob-hard3.txt','https://uchida.cis.k.hosei.ac.jp/2015/hard4.txt','https://uchida.cis.k.hosei.ac.jp/2015/hard5.txt']

for url in urls:
  grid = readSudoku(url)
  print('----------------------------------------------')
  print_grid()
  well_formed()
  timer = 0
  for i in range(30):
    start_time = time.time()
    solve()

  print(f'\n{timer/(i+1)}')

----------------------------------------------
9 2 1  | 7 3 5  | 4 8 6
4 7 6  | 2 8 9  | 5 1 3
5 8 3  | 1 6 4  | 2 7 9
- - - - - - - - - - - -
0 0 0  | 6 5 2  | 0 0 0
0 0 0  | 8 4 3  | 0 0 0
0 0 0  | 9 1 7  | 0 0 0
- - - - - - - - - - - -
2 4 7  | 5 9 8  | 3 6 1
6 5 9  | 3 2 1  | 7 4 8
1 3 8  | 4 7 6  | 9 2 5

Sudoku is well-formed (63 initial numbers).
It has a unique solution!

0.0022722800572713216
----------------------------------------------
5 0 0  | 3 6 0  | 0 0 1
2 0 4  | 0 0 0  | 0 3 0
0 7 0  | 0 9 0  | 0 0 4
- - - - - - - - - - - -
0 2 0  | 0 0 0  | 0 9 0
0 0 0  | 4 0 8  | 0 0 0
0 3 0  | 0 0 0  | 0 8 0
- - - - - - - - - - - -
3 0 0  | 0 8 0  | 0 1 0
0 8 0  | 0 0 0  | 7 0 6
7 0 0  | 0 4 5  | 0 0 8

Sudoku is well-formed (26 initial numbers).
It has a unique solution!

0.02442459265391032
----------------------------------------------
0 6 2  | 0 9 7  | 1 4 0
7 0 1  | 2 4 0  | 0 0 0
0 0 0  | 0 1 6  | 0 0 7
- - - - - - - - - - - -
1 7 0  | 0 0 0  | 0 0 4
3 8 0  | 4 0 1  | 0 7 0
0